In [25]:
# filter some warning messages
import warnings 
warnings.filterwarnings("ignore") 

#libraries
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# make datasets display nicely
xr.set_options(display_style="html")  
import os.path

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 


In [31]:
%%time

#Load in zarr of sattelite data
#satlist = ['goes-east','goes-west','himawari']
sat = 'goes_west'#choose a sattelite from satlist : 
file_location = 's3://ohw-bucket/'+sat

ds = xr.open_zarr(fsspec.get_mapper(file_location,anon=False))

#clean up variables that should be attributes
varlist = ['retrieval_solar_zenith_angle', 'day_solar_zenith_angle', 'y_image', 'retrieval_local_zenith_angle', 'x_image', 'night_solar_zenith_angle', 'quantitative_local_zenith_angle',
   'SST_night_only_emissive_wavelength', 'SST_night_only_emissive_band_id', 'SST_day_night_emissive_band_ids', 'SST_day_night_emissive_wavelengths']
for var in varlist:
    try : 
        ds[var]
    except KeyError: 
        pass
    else: 
        ds.attrs[var]=str(ds.variables[var].attrs['long_name']) + ' (' + str(ds.variables[var].attrs['units']) + ') = ' + str(ds.variables[var].values)
        ds = ds.drop(var, dim=None)

ds

ValueError: group not found at path ''

In [28]:
%%time
#Convert the cleaned up data to zarr
#sat = 'goes-east'
ds.to_zarr(sat, mode='w')
print(sat)

KeyboardInterrupt: 

In [29]:
#sync the locally stored zarr to remote server
#if only looking at 1 sattelite I would comment out the others for speed
! pip install awscli 
! aws s3 sync . s3://ohw-bucket/goes_east #/goes_east
! aws s3 sync . s3://ohw-bucket/goes_west #/goes_west
! aws s3 sync . s3://ohw-bucket/himawari #/goes_west

upload: himawari/.zattrs to s3://ohw-bucket/goes_west/himawari/.zattrs             
upload: himawari/.zgroup to s3://ohw-bucket/goes_west/himawari/.zgroup             
upload: himawari/DQF/.zarray to s3://ohw-bucket/goes_west/himawari/DQF/.zarray      
upload: himawari/DQF/.zattrs to s3://ohw-bucket/goes_west/himawari/DQF/.zattrs      
upload: himawari/DQF/103.0.0 to s3://ohw-bucket/goes_west/himawari/DQF/103.0.0        
upload: himawari/DQF/111.0.0 to s3://ohw-bucket/goes_west/himawari/DQF/111.0.0        
upload: himawari/DQF/23.0.0 to s3://ohw-bucket/goes_west/himawari/DQF/23.0.0          
upload: himawari/DQF/113.0.0 to s3://ohw-bucket/goes_west/himawari/DQF/113.0.0        
upload: himawari/DQF/0.0.0 to s3://ohw-bucket/goes_west/himawari/DQF/0.0.0            
upload: himawari/DQF/141.0.0 to s3://ohw-bucket/goes_west/himawari/DQF/141.0.0        
upload: himawari/DQF/45.0.0 to s3://ohw-bucket/goes_west/himawari/DQF/45.0.0          
upload: himawari/DQF/15.0.0 to s3://ohw-bucket/goes_w

In [30]:
os.getcwd()

'/home/jovyan/ohw20-proj-gapfree-sst/10-satellite-data-access'